In [1]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [3]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

In [4]:
df['point_bins'] = pd.cut(
    df['total_points_per_game'],
    [0, 30, 35, 40, 45, float("inf")],
    right=False,
    labels=['< 30', '30 - 35', '35 - 40', '40 - 45', "> 45"]
)

In [5]:
players_list = list(np.unique(df[['team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].values))

In [6]:
def get_player_stats(player, df):
    player_matches = df[
        np.where(
            np.logical_or.reduce([df[i] == player for i in ["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]]),
            True,
            False
        )
    ].copy()
    
    player_matches["belongs_to"] = np.where(
        np.logical_or(
            *[player_matches[i] == player for i in ["team_1_player_1", "team_1_player_2"]]
        ),
        'team_1',
        'team_2'
    )
    
    player_matches['player_team_points'] = np.where(
        player_matches["belongs_to"] == 'team_1',
        player_matches["points_team_1"],
        player_matches["points_team_2"]
    )
    
    player_matches['result'] = np.where(player_matches.belongs_to == player_matches.winner, "win", "loss")
    player_matches['is_win'] = np.where(player_matches.result == "win", 1, 0)
    
    return player_matches

In [7]:
team_1 = ['raghotham', 'nithin']
team_2 = ['ajay', 'vinay']

In [12]:
player_matches = get_player_stats(team_1[0], df)
player_matches['partner'] = np.where(
    player_matches["belongs_to"] == 'team_1',
    np.where(
        player_matches["team_1_player_1"] == team_1[0],
        player_matches["team_1_player_2"],
        player_matches["team_1_player_1"]
    ),
    np.where(
        player_matches["team_2_player_1"] == team_1[0],
        player_matches["team_2_player_2"],
        player_matches["team_2_player_1"]
    ),
)

In [60]:
head_2_head_df = player_matches[np.where(
    player_matches['belongs_to'] == 'team_1',
    np.logical_and.reduce([
        player_matches['partner'] == team_1[1],
        player_matches['team_2_player_1'].isin(team_2),
        player_matches['team_2_player_2'].isin(team_2),
    ]),
    np.logical_and.reduce([
        player_matches['partner'] == team_1[1],
        player_matches['team_1_player_1'].isin(team_2),
        player_matches['team_1_player_2'].isin(team_2),
    ])
)].copy().reset_index()

head_2_head_df["other_team_points"] = head_2_head_df["total_points_per_game"] - head_2_head_df["player_team_points"]

In [89]:
head_2_head_df

,index,date,team_1_player_1,team_1_player_2,team_2_player_1,team_2_player_2,points_team_1,points_team_2,venue,winner,margin,total_points_per_game,point_bins,belongs_to,player_team_points,result,is_win,partner,other_team_points
0,16,12/30/2022,nithin,raghotham,ajay,vinay,14,21,prakash badminton academy,team_2,7,35,35 - 40,team_1,14,loss,0,nithin,21
1,18,12/30/2022,nithin,raghotham,ajay,vinay,14,21,prakash badminton academy,team_2,7,35,35 - 40,team_1,14,loss,0,nithin,21
2,20,12/30/2022,nithin,raghotham,ajay,vinay,20,22,prakash badminton academy,team_2,2,42,40 - 45,team_1,20,loss,0,nithin,22
3,22,12/30/2022,nithin,raghotham,ajay,vinay,14,21,prakash badminton academy,team_2,7,35,35 - 40,team_1,14,loss,0,nithin,21
4,36,12/26/2022,nithin,raghotham,ajay,vinay,21,15,match point - gublaala,team_1,6,36,35 - 40,team_1,21,win,1,nithin,15
5,37,12/26/2022,nithin,raghotham,ajay,vinay,18,21,match point - gublaala,team_2,3,39,35 - 40,team_1,18,loss,0,nithin,21
6,38,12/26/2022,nithin,raghotham,ajay,vinay,16,21,match point - gublaala,team_2,5,37,35 - 40,team_1,16,loss,0,nithin,21
7,41,12/26/2022,nithin,raghotham,ajay,vinay,21,13,match point - gublaala,team_1,8,34,30 - 35,team_1,21,win,1,nithin,13


In [62]:
longest_game = head_2_head_df.iloc[head_2_head_df["total_points_per_game"].idxmax(), :].to_dict()
longest_game

{'index': 20,
 'date': '12/30/2022',
 'team_1_player_1': 'nithin',
 'team_1_player_2': 'raghotham',
 'team_2_player_1': 'ajay',
 'team_2_player_2': 'vinay',
 'points_team_1': 20,
 'points_team_2': 22,
 'venue': 'prakash badminton academy',
 'winner': 'team_2',
 'margin': 2,
 'total_points_per_game': 42,
 'point_bins': '40 - 45',
 'belongs_to': 'team_1',
 'player_team_points': 20,
 'result': 'loss',
 'is_win': 0,
 'partner': 'nithin',
 'other_team_points': 22}

In [155]:
def get_game_result_string(game):
    return f"{game['total_points_per_game']} points: ({game['team_1_player_1']}, {game['team_1_player_2']}) {game['points_team_1']} - {game['points_team_2']} ({game['team_2_player_1']}, {game['team_2_player_2']}) on {game['date']} at {game['venue']}"

head_2_head_stats = {
    "total_games": head_2_head_df.shape[0],
    "team_1_wins": head_2_head_df["is_win"].sum(),
    "average_ppg": head_2_head_df["total_points_per_game"].mean(),
    "average_margin_of_victory": head_2_head_df["margin"].mean(),
    "avg_team1_pts": head_2_head_df["player_team_points"].mean(),
    "avg_team2_pts": head_2_head_df["other_team_points"].mean(),
    "longest_game": get_game_result_string(longest_game)
}

head_2_head_stats["team_2_wins"] = head_2_head_stats["total_games"] - head_2_head_stats["team_1_wins"]

head_2_head_stats["team_1_avg_win_margin"] = round(head_2_head_df[head_2_head_df["is_win"] == 1]["margin"].mean(), 2)
head_2_head_stats["team_2_avg_win_margin"] = round(head_2_head_df[head_2_head_df["is_win"] == 0]["margin"].mean(), 2)

head_2_head_stats["team_1_min_points_in_game"] = head_2_head_df["player_team_points"].min()
head_2_head_stats["team_2_min_points_in_game"] = head_2_head_df["other_team_points"].min()

head_2_head_stats["team_1_largest_win"] = get_game_result_string(head_2_head_df.iloc[head_2_head_df[head_2_head_df["is_win"] == 1]["margin"].idxmax(), :].to_dict())
head_2_head_stats["team_2_largest_win"] = get_game_result_string(head_2_head_df.iloc[head_2_head_df[head_2_head_df["is_win"] == 0]["margin"].idxmax(), :].to_dict())

In [167]:
head_2_head_stats["team_1_games_won_after_deuce"] = head_2_head_df[(head_2_head_df["total_points_per_game"] > 40) & (head_2_head_df["is_win"] == 1)].shape[0]
head_2_head_stats["team_2_games_won_after_deuce"] = head_2_head_df[(head_2_head_df["total_points_per_game"] > 40) & (head_2_head_df["is_win"] == 0)].shape[0]

In [161]:
head_2_head_stats["games_gone_beyond_deuces"] = (head_2_head_df["total_points_per_game"] > 40).sum()

In [168]:
head_2_head_stats

{'total_games': 8,
 'team_1_wins': 2,
 'average_ppg': 36.625,
 'average_margin_of_victory': 5.625,
 'avg_team1_pts': 17.25,
 'avg_team2_pts': 19.375,
 'longest_game': '42 points: (nithin, raghotham) 20 - 22 (ajay, vinay) on 12/30/2022 at prakash badminton academy',
 'team_2_wins': 6,
 'team_1_avg_win_margin': 7.0,
 'team_2_avg_win_margin': 5.17,
 'team_1_min_points_in_game': 14,
 'team_2_min_points_in_game': 13,
 'team_1_largest_win': '34 points: (nithin, raghotham) 21 - 13 (ajay, vinay) on 12/26/2022 at match point - gublaala',
 'team_2_largest_win': '35 points: (nithin, raghotham) 14 - 21 (ajay, vinay) on 12/30/2022 at prakash badminton academy',
 'games_gone_beyond_deuces': 1,
 'team_1_games_won_after_deuce': 0,
 'team_2_games_won_after_deuce': 1}

In [169]:
df2 = pd.DataFrame([head_2_head_stats]).T
df2

,0
total_games,8
team_1_wins,2
average_ppg,36.625
average_margin_of_victory,5.625
avg_team1_pts,17.25
avg_team2_pts,19.375
longest_game,"42 points: (nithin, raghotham) 20 - 22 (ajay, ..."
team_2_wins,6
team_1_avg_win_margin,7.0
team_2_avg_win_margin,5.17


In [197]:
comparision_table_list = [
    ["Wins", "Average points per game", "Average Win Margin", "Minimum Points in a Game", "Games Won post Deuce", "Largest Win"],
    [head_2_head_stats["team_1_wins"], head_2_head_stats["avg_team1_pts"], head_2_head_stats["team_1_avg_win_margin"], head_2_head_stats["team_1_min_points_in_game"], head_2_head_stats["team_1_games_won_after_deuce"], head_2_head_stats["team_1_largest_win"]],
    [head_2_head_stats["team_2_wins"], head_2_head_stats["avg_team2_pts"], head_2_head_stats["team_2_avg_win_margin"], head_2_head_stats["team_2_min_points_in_game"], head_2_head_stats["team_2_games_won_after_deuce"], head_2_head_stats["team_2_largest_win"]]
]

In [199]:
cell_colors = [
    ['#eceff1'] * len(comparision_table_list[0]),
    [('#eceff1' if comparision_table_list[1][i] < comparision_table_list[2][i] else '#d8ee91') for i in range(len(comparision_table_list[0]) - 1)] + ['#eceff1'],
    [('#eceff1' if comparision_table_list[2][i] < comparision_table_list[1][i] else '#d8ee91') for i in range(len(comparision_table_list[0]) - 1)] + ['#eceff1']
]
cell_colors

[['#eceff1', '#eceff1', '#eceff1', '#eceff1', '#eceff1', '#eceff1'],
 ['#eceff1', '#eceff1', '#d8ee91', '#d8ee91', '#eceff1', '#eceff1'],
 ['#d8ee91', '#d8ee91', '#eceff1', '#eceff1', '#d8ee91', '#eceff1']]

In [200]:
table = go.Table(
    header=dict(
        values=["", " & ".join(team_1), " & ".join(team_2)], 
        align="left", 
        height=40, 
        font=dict(color="white", size=16),
        fill_color="lightslategrey"
    ),
    cells=dict(
        values=comparision_table_list,
        align="left",
        height=60,
        font=dict(size=14),
        fill_color=cell_colors
    )
)
fig = go.Figure(table)
fig

In [88]:
table = go.Table(
    columnwidth=[2, 3],
    header=dict(values=["Metric", "Detail"], align="left", height=40, font=dict(size=20)),
    cells=dict(
        values=[list(head_2_head_stats.keys()), list(head_2_head_stats.values())],
        align="left",
        height=40,
        font=dict(size=15)
    )
)
fig = go.Figure(table)
fig

In [98]:
go.Figure(
    go.Pie(
        labels=[' & '.join(team_1), ' & '.join(team_2)],
        values=[head_2_head_stats["team_1_wins"], head_2_head_stats["team_2_wins"]],
        hole=0.3,
        marker=dict(colors=["#b5de2b", "lightslategrey"])
    ),
)

In [131]:
fig = go.Figure(
    go.Indicator(
        mode="gauge+number+delta",
        value=head_2_head_stats['average_ppg'],
        title="Average Points per Game",
        delta={"reference": 38.3}
    )
)

fig.update_traces(gauge_bar_color="#66bb6a")